In [3]:
import pandas as pd
path = '/rds/general/user/ms2524/home/amexproject/parquet_files/train/train_'
base_cba = pd.read_parquet(path + "base.parquet", columns=['case_id','WEEK_NUM','target'])

In [1]:
import pandas as pd

def missing_value_percentages(df):
    missing_percent = df.isnull().mean() * 100
    return missing_percent.sort_values(ascending=False)
def detect_high_cardinality(df, threshold=50):
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    high_card = [col for col in cat_cols if df[col].nunique() > threshold]
    return high_card

# Debitcard, Deposit and other table

In [3]:
debitcard_1 = pd.read_parquet(path +'debitcard_1.parquet')
deposit_1 = pd.read_parquet(path +'deposit_1.parquet')
other_1 = pd.read_parquet(path +'other_1.parquet')

### Not all ids appear in these tables, its a small subset of ids so is it worth using these tables

# Deposit

In [13]:
deposit_1["case_id"].nunique()

105111

In [20]:
deposit_1

,case_id,amount_416A,contractenddate_991D,num_group1,openingdate_313D
0,225,0.00000,NaT,0,2016-08-16
1,331,260.37400,2018-03-18,0,2015-03-19
2,358,0.00000,NaT,0,2014-09-02
3,390,203.60200,2017-09-30,1,2015-10-01
4,390,223.68001,NaT,2,2016-06-08
...,...,...,...,...,...
145081,2703430,0.00000,NaT,7,2016-02-25
145082,2703430,22918.00600,NaT,8,2016-11-26
145083,2703439,219.98401,NaT,0,2016-11-25
145084,2703453,0.00000,NaT,0,2014-08-18


In [19]:
import pandas as pd

def process_deposit_accounts(df):
    df['contractenddate_991D'] = pd.to_datetime(df['contractenddate_991D'], errors='coerce')
    def custom_agg(group):
        total_amount = group['amount_416A'].sum()
        contract_closed_flag = int(group['contractenddate_991D'].notna().all())
        return pd.Series({
            'amount_416A': total_amount,
            'is_closed': contract_closed_flag
        })

    result = df.groupby('case_id').apply(custom_agg).reset_index()
    return result
process_deposit_accounts(deposit_1)

/rds/general/user/ms2524/ephemeral/ipykernel_367002/3904144058.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby('case_id').apply(custom_agg).reset_index()


,case_id,amount_416A,is_closed
0,225,0.00000,0.0
1,331,260.37400,1.0
2,358,0.00000,0.0
3,390,212175.81201,0.0
4,445,23735.93800,0.0
...,...,...,...
105106,2703426,1181.73610,1.0
105107,2703427,250.76399,1.0
105108,2703430,22918.00600,0.0
105109,2703439,219.98401,0.0


# Debitcard

In [14]:
debitcard_1["case_id"].nunique()

111772

In [23]:
debitcard_1[debitcard_1["openingdate_857D"].isnull()]

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A,num_group1,openingdate_857D
35,1661,NaN,NaN,NaN,0,None
36,1661,NaN,NaN,NaN,1,None
37,1661,NaN,NaN,NaN,2,None
38,1661,NaN,NaN,NaN,3,None
43,1887,NaN,NaN,NaN,0,None
...,...,...,...,...,...,...
157242,2703197,1.880000,12607.4,0.0,0,None
157246,2703220,0.000000,23800.0,0.0,0,None
157254,2703297,0.000000,71000.0,0.0,0,None
157261,2703329,0.633334,29997.0,0.0,0,None


In [24]:
missing_value_percentages(debitcard_1)

last30dayturnover_651A           92.955589
last180dayturnover_1134A         92.955589
last180dayaveragebalance_704A    92.234047
openingdate_857D                  8.080635
case_id                           0.000000
num_group1                        0.000000
dtype: float64

### We will drop the time feature 'openingdate_857D' and aggregate the remaining feaatures across ids.

In [31]:
debitcard_1.drop(columns='num_group1').groupby('case_id', as_index=False).sum(numeric_only=True)

,case_id,last180dayaveragebalance_704A,last180dayturnover_1134A,last30dayturnover_651A
0,225,0.0,0.0,0.0
1,331,0.0,0.0,0.0
2,358,0.0,0.0,0.0
3,390,0.0,0.0,0.0
4,445,0.0,0.0,0.0
...,...,...,...,...
111767,2703426,0.0,0.0,0.0
111768,2703427,0.0,0.0,0.0
111769,2703430,0.0,0.0,0.0
111770,2703439,0.0,0.0,0.0


## Other

In [21]:
other_1["case_id"].nunique()

51109

In [26]:
other_1.drop(columns='num_group1').groupby('case_id', as_index=False).sum(numeric_only=True)


,case_id,amtdebitincoming_4809443A,amtdebitoutgoing_4809440A,amtdepositbalance_4809441A,amtdepositincoming_4809444A,amtdepositoutgoing_4809442A
0,43801,12466.6010,12291.2000,914.2000,0.0,304.80002
1,43991,3333.4001,3273.4001,0.0000,0.0,0.00000
2,44001,10000.0000,10000.0000,0.0000,0.0,0.00000
3,44053,0.0000,0.0000,2586.4001,0.0,88.80000
4,44130,63.8000,60.8000,0.0000,0.0,0.00000
...,...,...,...,...,...,...
51104,2703443,344.6000,343.4000,0.0000,0.0,0.00000
51105,2703448,0.0000,0.0000,2184.2000,0.0,18.00000
51106,2703450,0.0000,0.0000,0.0000,0.0,0.00000
51107,2703451,27500.0000,27477.6000,0.0000,0.0,0.00000


# Person

In [35]:
person_1 = pd.read_parquet(path +'person_1.parquet')
person_2 = pd.read_parquet(path +'person_2.parquet')

In [36]:
person_1

,case_id,birth_259D,birthdate_87D,childnum_185L,contaddr_district_15M,contaddr_matchlist_1032L,contaddr_smempladdr_334L,contaddr_zipcode_807M,education_927M,empl_employedfrom_271D,...,registaddr_district_1083M,registaddr_zipcode_184M,relationshiptoclient_415T,relationshiptoclient_642T,remitter_829L,role_1084L,role_993L,safeguarantyflag_411L,sex_738L,type_25L
0,0,1986-07-01,None,NaN,P88_18_84,False,False,P167_100_165,P97_36_170,2017-09-15,...,P88_18_84,P167_100_165,None,None,None,CL,None,True,F,PRIMARY_MOBILE
1,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,SPOUSE,None,False,EM,None,None,None,PHONE
2,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,COLLEAGUE,SPOUSE,False,PE,None,None,None,PHONE
3,0,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,COLLEAGUE,None,PE,None,None,None,PHONE
4,1,1957-08-01,None,NaN,P103_93_94,False,False,P176_37_166,P97_36_170,2008-10-29,...,P103_93_94,P176_37_166,None,None,None,CL,None,True,M,PRIMARY_MOBILE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2973986,2703451,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,None,None,CL,None,None,None,HOME_PHONE
2973987,2703452,1977-08-01,None,NaN,P133_44_167,False,False,P59_150_74,a55475b1,None,...,P19_11_176,P11_15_81,None,None,None,CL,None,False,M,PRIMARY_MOBILE
2973988,2703453,1950-02-01,None,NaN,P123_6_84,False,False,P46_103_143,a55475b1,None,...,P123_6_84,P46_103_143,None,None,None,CL,None,False,F,PRIMARY_MOBILE
2973989,2703453,None,None,NaN,a55475b1,None,None,a55475b1,a55475b1,None,...,a55475b1,a55475b1,None,None,None,CL,None,None,None,HOME_PHONE


In [38]:
missing_value_percentages(person_2)

empls_employedfrom_796D     99.649692
relatedpersons_role_762T    98.252049
addres_role_871L            95.882099
addres_district_368M         0.000000
case_id                      0.000000
conts_role_79M               0.000000
addres_zip_823M              0.000000
empls_economicalst_849M      0.000000
empls_employer_name_740M     0.000000
num_group1                   0.000000
num_group2                   0.000000
dtype: float64

In [45]:
person_1["case_id"].nunique()

1526659

In [ ]:
person_1[person_1["num_group1"]==0]["familystate_447L"].value_counts(dropna=False)

In [46]:
missing_value_percentages(person_1[person_1["num_group1"]==0])

childnum_185L                100.000000
relationshiptoclient_642T    100.000000
remitter_829L                100.000000
relationshiptoclient_415T    100.000000
housingtype_772L             100.000000
maritalst_703L                99.999934
gender_992L                   99.183970
isreference_387L              99.183970
birthdate_87D                 99.183970
role_993L                     99.183970
housetype_905L                93.396168
empl_industry_691L            65.792230
empl_employedtotal_800L       65.394040
empl_employedfrom_271D        62.879661
familystate_447L              52.274018
contaddr_smempladdr_334L       0.028887
contaddr_matchlist_1032L       0.028887
safeguarantyflag_411L          0.000131
contaddr_district_15M          0.000000
case_id                        0.000000
birth_259D                     0.000000
language1_981M                 0.000000
incometype_1044T               0.000000
empladdr_zipcode_114M          0.000000
empladdr_district_926M         0.000000


In [55]:
detect_high_cardinality(person_1[person_1["num_group1"]==0])

['birth_259D',
 'birthdate_87D',
 'contaddr_district_15M',
 'contaddr_zipcode_807M',
 'empl_employedfrom_271D',
 'empladdr_district_926M',
 'empladdr_zipcode_114M',
 'registaddr_district_1083M',
 'registaddr_zipcode_184M']

Feature                      Missing amount

childnum_185L                100.000000
relationshiptoclient_642T    100.000000
remitter_829L                100.000000
relationshiptoclient_415T    100.000000
housingtype_772L             100.000000
maritalst_703L                99.999934
gender_992L                   99.183970
isreference_387L              99.183970
birthdate_87D                 99.183970
role_993L                     99.183970
housetype_905L                93.396168

High cardinality features:

empladdr_zipcode_114M          0.000000
empladdr_district_926M         0.000000
contaddr_zipcode_807M          0.000000
registaddr_zipcode_184M        0.000000
registaddr_district_1083M      0.000000


These features just have one value so theres no value in keeping them:

persontype_792L                0.000000
persontype_1072L               0.000000
personindex_1023L              0.000000



empl_industry_691L            65.792230 - High Cardinality
empl_employedtotal_800L       65.394040 - Useful features
empl_employedfrom_271D        62.879661 - Similar to empl_employedtotal_800L 



Selected features:
empl_employedtotal_800L 
familystate_447L
contaddr_smempladdr_334L       0.028887
contaddr_matchlist_1032L       0.028887
safeguarantyflag_411L          0.000131
birth_259D                     0.000000 convert to age
language1_981M                 0.000000
incometype_1044T               0.000000
education_927M                 0.000000
mainoccupationinc_384A         0.000000
role_1084L                     0.000000
sex_738L                       0.000000
type_25L                       0.000000

In [90]:
missing_value_percentages(person_2[person_2["num_group1"]==0])

empls_employedfrom_796D     99.999932
relatedpersons_role_762T    99.999932
addres_role_871L            98.859917
addres_district_368M         0.000000
case_id                      0.000000
conts_role_79M               0.000000
addres_zip_823M              0.000000
empls_economicalst_849M      0.000000
empls_employer_name_740M     0.000000
num_group1                   0.000000
num_group2                   0.000000
dtype: float64

In [99]:
detect_high_cardinality(person_2[(person_2["num_group1"]==0) & (person_2["num_group2"]==0)])

['addres_district_368M', 'addres_zip_823M']

In [92]:
person_2["case_id"].nunique()

1435105

In [97]:
person_1["case_id"].nunique()

1526659

In [95]:
person_2[person_2["num_group1"]==0]["empls_employer_name_740M"].value_counts(dropna=False)

empls_employer_name_740M
a55475b1      1463927
P207_71_72          1
Name: count, dtype: int64

High cardinality features:
'addres_district_368M', 'addres_zip_823M'

High missing features:
empls_employedfrom_796D     99.999932
relatedpersons_role_762T    99.999932
addres_role_871L            98.859917


Useless features:
empls_employer_name_740M - just 1 value for second category

Features elected:
empls_economicalst_849M
conts_role_79M

# Previous applications

In [4]:
applprev_1_0 = pd.read_parquet(path +'applprev_1_0.parquet')
applprev_1_1 = pd.read_parquet(path +'applprev_1_1.parquet')
applprev_2 = pd.read_parquet(path + 'applprev_2.parquet')

In [5]:
applprev = pd.concat([applprev_1_0, applprev_1_1])

In [6]:
missing_value_percentages(applprev[applprev["num_group1"]==0])

credacc_actualbalance_314A     94.641030
credacc_transactions_402L      94.641030
credacc_status_367L            94.641030
credacc_minhisbal_90A          94.641030
credacc_maxhisbal_375A         94.641030
byoccupationinc_3656910L       94.015089
revolvingaccount_394A          90.685063
isdebitcard_527L               86.404666
childnum_21L                   83.165510
dtlastpmt_581D                 77.041101
employedfrom_700D              71.410175
familystate_726L               59.084650
dtlastpmtallstes_3545839D      47.170006
dateactivated_425D             43.019528
maxdpdtolerance_577P           42.240827
approvaldate_319D              38.819604
outstandingdebt_522A           38.689275
currdebt_94A                   38.381462
firstnonzeroinstldate_307D     15.794067
tenor_203L                     11.152808
pmtnum_8L                      11.152808
mainoccupationinc_437A          3.433094
annuity_853A                    2.702121
inittransactioncode_279L        2.328161
credamount_590A 

In [106]:

detect_high_cardinality(applprev)

['approvaldate_319D',
 'cancelreason_3545846M',
 'creationdate_885D',
 'dateactivated_425D',
 'district_544M',
 'dtlastpmt_581D',
 'dtlastpmtallstes_3545839D',
 'employedfrom_700D',
 'firstnonzeroinstldate_307D',
 'profession_152M']

In [8]:
applprev["inittransactioncode_279L"].value_counts(dropna=False)

inittransactioncode_279L
POS     3501653
CASH    2576770
NDF      245358
None     202198
Name: count, dtype: int64

In [119]:
applprev_2[(applprev_2["num_group1"]==0)&(applprev_2["num_group2"]==0)]

,case_id,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group1,num_group2
0,2,None,PRIMARY_MOBILE,None,0,0
4,3,None,PHONE,None,0,0
7,4,None,PRIMARY_MOBILE,None,0,0
9,5,None,PRIMARY_MOBILE,None,0,0
10,6,None,PRIMARY_MOBILE,None,0,0
...,...,...,...,...,...,...
14075405,2703450,a55475b1,PRIMARY_MOBILE,None,0,0
14075434,2703451,a55475b1,PRIMARY_MOBILE,None,0,0
14075451,2703452,a55475b1,PRIMARY_MOBILE,None,0,0
14075457,2703453,a55475b1,PRIMARY_MOBILE,None,0,0


Date features to not be used:

dtlastpmt_581D                 72.791898
dtlastpmtallstes_3545839D      61.961906
employedfrom_700D              59.553946
dateactivated_425D             48.142248
approvaldate_319D              38.819604
firstnonzeroinstldate_307D      9.998224
creationdate_885D               0.000000

Features with high number of missing:

revolvingaccount_394A          95.452790
credacc_minhisbal_90A          95.096000
credacc_status_367L            95.096000
credacc_maxhisbal_375A         95.096000
credacc_transactions_402L      95.096000
credacc_actualbalance_314A     95.096000
isdebitcard_527L               92.905080
byoccupationinc_3656910L       76.487083

Features to use:

childnum_21L                   54.542376
maxdpdtolerance_577P           47.436622 (median or mode)
familystate_726L               36.682496 (Categorical unknown)
outstandingdebt_522A           34.604279 (impute both debts as 0?)
currdebt_94A                   34.424138 (impute both debts as 0?)
tenor_203L                      8.455743
pmtnum_8L                       8.455743 (0 most common use median or mode) this is the same as tenor it seems
annuity_853A                    3.842121
credamount_590A                 3.098355
credtype_587L                   3.098355
downpmt_134A                    3.098355
inittransactioncode_279L        3.098355
credacc_credlmt_575A            2.974757
mainoccupationinc_437A          1.562723
actualdpd_943P                  0.038308 (0 most common)
status_219L                     0.000000
isbidproduct_390L               0.000000

cancelreason_3545846M           0.000000 
district_544M                   0.000000
education_1138M                 0.000000
rejectreason_755M               0.000000
profession_152M                 0.000000
postype_4733339M                0.000000
rejectreasonclient_4145042M     0.000000

In [49]:
applprev_2[(applprev_2["num_group1"]==0)&(applprev_2["num_group2"]==0)]["credacc_cards_status_52L"].value_counts(dropna=False)

credacc_cards_status_52L
None           1156883
CANCELLED        37144
ACTIVE           15808
INACTIVE         11331
BLOCKED            293
RENEWED             59
UNCONFIRMED          4
Name: count, dtype: int64

In [48]:
applprev_2['credacc_cards_status_52L'].unique()


array([None, 'CANCELLED', 'INACTIVE', 'ACTIVE', 'BLOCKED', 'RENEWED',
       'UNCONFIRMED'], dtype=object)

In [40]:
applprev_2

,case_id,cacccardblochreas_147M,conts_type_509L,credacc_cards_status_52L,num_group1,num_group2
0,2,None,PRIMARY_MOBILE,None,0,0
1,2,None,EMPLOYMENT_PHONE,None,0,1
2,2,None,PRIMARY_MOBILE,None,1,0
3,2,None,EMPLOYMENT_PHONE,None,1,1
4,3,None,PHONE,None,0,0
...,...,...,...,...,...,...
14075482,2703454,a55475b1,None,None,0,1
14075483,2703454,a55475b1,PRIMARY_MOBILE,None,1,0
14075484,2703454,a55475b1,HOME_PHONE,None,1,1
14075485,2703454,a55475b1,None,None,1,2


Features:

cacccardblochreas_147M
conts_type_509L
credacc_cards_status_52L

# Static

In [7]:
static_0_0 =  pd.read_parquet(path +'static_0_0.parquet')
static_0_1 = pd.read_parquet(path +'static_0_1.parquet')

static = pd.concat([static_0_0, static_0_1])

In [53]:
static

,case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,...,sellerplacecnt_915L,sellerplacescnt_216L,sumoutstandtotal_3546847A,sumoutstandtotalest_4493215A,totaldebt_9A,totalsettled_863A,totinstallast1m_4525188A,twobodfilling_608L,typesuite_864L,validfrom_1069D
0,0,NaN,NaN,1917.6000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,None,None
1,1,NaN,NaN,3134.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,None,None
2,2,NaN,NaN,4937.0000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,AL,None
3,3,NaN,NaN,4643.6000,0.0,0.0,1.0,0.0,2.0,0.0,...,1.0,1.0,NaN,NaN,0.0,0.00,NaN,BO,AL,None
4,4,NaN,NaN,3390.2000,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,NaN,NaN,0.0,0.00,NaN,BO,AL,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522897,2703450,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8.0,0.0,0.0,0.0,428159.66,14346.319,FO,None,None
522898,2703451,0.0,301276.47,7088.6000,6191.6,0.0,0.0,5.0,0.0,0.0,...,0.0,3.0,68098.4,68098.4,68098.4,701247.30,40499.805,FO,None,None
522899,2703452,0.0,14232.40,7788.8003,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,24002.00,NaN,BO,None,None
522900,2703453,0.0,197371.58,1195.4000,2827.2,0.0,0.0,36.0,0.0,0.0,...,0.0,4.0,46806.6,46806.6,46806.6,440145.30,5654.400,BO,None,None


In [9]:
missing_value_percentages(static)


clientscnt_136L                 99.972685
lastrepayingdate_696D           99.844301
lastotherinc_902A               99.799824
lastotherlnsexpense_631A        99.799693
payvacationpostpone_4187118D    99.388927
                                  ...    
applicationscnt_464L             0.000000
previouscontdistrict_112M        0.000000
case_id                          0.000000
sellerplacecnt_915L              0.000000
sellerplacescnt_216L             0.000000
Length: 168, dtype: float64

In [10]:
detect_high_cardinality(static)

['datefirstoffer_1144D',
 'datelastinstal40dpd_247D',
 'datelastunpaid_3546854D',
 'dtlastpmtallstes_4499206D',
 'firstclxcampaign_1125D',
 'firstdatedue_489D',
 'lastactivateddate_801D',
 'lastapplicationdate_877D',
 'lastapprcommoditytypec_5251766M',
 'lastapprdate_640D',
 'lastcancelreason_561M',
 'lastdelinqdate_224D',
 'lastrejectcommodtypec_5251769M',
 'lastrejectdate_50D',
 'lastrepayingdate_696D',
 'maxdpdinstldate_3546855D',
 'payvacationpostpone_4187118D',
 'previouscontdistrict_112M',
 'validfrom_1069D']

In [11]:
static["amtinstpaidbefduel24m_4187115A"].value_counts(dropna=False)

amtinstpaidbefduel24m_4187115A
NaN           561124
0.000         161063
800.000           89
6000.000          84
4000.000          79
               ...  
36483.200          1
75629.586          1
154279.610         1
34429.688          1
134850.200         1
Name: count, Length: 610153, dtype: int64




missing:
clientscnt_136L
cardtype_51L
bankacctype_710L
isbidproductrequest_292L
isdebitcard_729L
interestrategrace_34L
inittransactionamount_650A
equalityempfrom_62L
equalitydataagreement_891L
lastotherinc_902A
lastdependentsnum_448L
totinstallast1m_4525188A
sumoutstandtotalest_4493215A

avglnamtstart24m_4525187A
avgoutstandbalancel6m_4187114A
avgpmtlast12m_4525200A
avgdbdtollast24m_4525197P
avgdbddpdlast3m_4187120P
maxoutstandbalancel12m_4187113A
maxpmtlast3m_4525190A
maxlnamtstart6m_4525199A
validfrom_1069D


datelastinstal40dpd_247D
dtlastpmtallstes_4499206D
datelastunpaid_3546854D
lastrepayingdate_696D
